This is a test to check whether my algorithm works accurate or not using the example given in the markov decision processes for value iteration algorithm.

#value check

In [ ]:
r = 0 #live-in reward
R1 = 1 #exit reward
R2 = -1 #hazard reward
D, H = (4,3), (4,2) #destination, hazard
U = {(col, row):0 for col in range(1,5) for row in range(1,4)}
U_new = {(col, row):0 for col in range(1,5) for row in range(1,4)}
reward = {(col, row):r for col in range(1,5) for row in range(1,4)}
U[D]= R1
U[H]= R2
U_new[D]= R1
U_new[H]= R2
reward[D]= R1
reward[H]= R2
obstacle = (2,2)
U[obstacle] = None
U_new[obstacle] = None
reward[obstacle] = None
print(U)
print(U_new)
print(reward)

{(1, 1): 0, (1, 2): 0, (1, 3): 0, (2, 1): 0, (2, 2): None, (2, 3): 0, (3, 1): 0, (3, 2): 0, (3, 3): 0, (4, 1): 0, (4, 2): -1, (4, 3): 1}
{(1, 1): 0, (1, 2): 0, (1, 3): 0, (2, 1): 0, (2, 2): None, (2, 3): 0, (3, 1): 0, (3, 2): 0, (3, 3): 0, (4, 1): 0, (4, 2): -1, (4, 3): 1}
{(1, 1): 0, (1, 2): 0, (1, 3): 0, (2, 1): 0, (2, 2): None, (2, 3): 0, (3, 1): 0, (3, 2): 0, (3, 3): 0, (4, 1): 0, (4, 2): -1, (4, 3): 1}


In [ ]:
def update_directions(directions,obstacle,col_size,row_size,curr_state):
  for i in range(len(directions)):
    if (directions[i] == obstacle) or (directions[i][0] > col_size or directions[i][1] > row_size) or (directions[i][0] < 1 or directions[i][1] < 1):
      directions[i] = curr_state
    # if directions[i][0] > col_size or directions[i][1] > row_size:
    #   directions[i] = None
  return directions

def pattern_order(destination=(4,3), obstacle=(2,2), col_size=4, row_size=3, hazard=(4,2)):
  visited_nodes = []
  curr_child_nodes = [destination]
  child_nodes_order = []
  while curr_child_nodes:
    nxt_list = []
    for child in curr_child_nodes:
      col, row = child
      next_children = [(col+1,row),(col, row+1),(col-1, row),(col, row-1)]
      possible_nxt_child = list(set(update_directions(next_children,obstacle,col_size,row_size,(col,row))))
      for node in possible_nxt_child:
        nxt_list.append(node) if (node not in visited_nodes and node not in [destination, hazard]) else nxt_list
        visited_nodes.append(node) if (node not in visited_nodes and node not in [destination,hazard]) else visited_nodes
    child_nodes_order.append(nxt_list)
    curr_child_nodes = nxt_list
    # print(curr_child_nodes)
  return visited_nodes, child_nodes_order


In [ ]:
u, v = pattern_order((4,3),(2,2),4,3,(4,2))
print(u)
print(v)

[(3, 3), (2, 3), (3, 2), (1, 3), (3, 1), (1, 2), (4, 1), (2, 1), (1, 1)]
[[(3, 3)], [(2, 3), (3, 2)], [(1, 3), (3, 1)], [(1, 2), (4, 1), (2, 1)], [(1, 1)], []]


In [ ]:
def max_prob(state, U, col_size, row_size, obstacle):
  col, row = state
  opp_directions = {'up':'down', 'down':'up', 'right':'left', 'left':'right'}
  new_directions = [(col+1,row),(col, row+1),(col-1, row),(col, row-1)] #right, up, left, down
  directions = update_directions(new_directions,obstacle,col_size,row_size,(col,row))
  intended_directions = ['up', 'down', 'left', 'right']
  val = 0
  curr_right,curr_up,curr_left,curr_down  = directions[0], directions[1], directions[2], directions[3]
  for direction in intended_directions:
    if direction == 'up':
      curr_utility = 0.8*U[curr_up] + 0.1*U[curr_left] + 0.1*U[curr_right]
      val = max(val, curr_utility)
    elif direction == 'down':
      curr_utility = 0.8*U[curr_down] + 0.1*U[curr_left] + 0.1*U[curr_right]
      val = max(val, curr_utility)
    elif direction == 'left':
      curr_utility = 0.8*U[curr_left] + 0.1*U[curr_up] + 0.1*U[curr_down]
      val = max(val, curr_utility)
    elif direction == 'right':
      curr_utility = 0.8*U[curr_right] + 0.1*U[curr_up] + 0.1*U[curr_down]
      val = max(val, curr_utility)
  return val

In [ ]:
import copy
col_size, row_size = 4,3
def valueIteration(threshold_value=0.01, max_iter=9, gamma=0.9, reward=reward, U=U, obstacle=obstacle, U_new=U_new,col_size=4, row_size=3, destination=(4,3), hazard=(4,2)):
    #
  # U = {(col, row): 0 for col in range(1,6) for row in range(1,6)}
  # for _ in range(max_iter):
    # U_new = copy.deepcopy(U)
    curr_seen = seen = [destination]
    k = 1
    not_threshold_condition = True
    all_states_order, child_nodes_order = pattern_order(destination,obstacle,col_size,row_size,hazard)
    while not_threshold_condition and k < max_iter:
        k += 1
      # if k <= len(child_nodes_order):
      #   for child in child_nodes_order[:k]:
      #     # print(child)
      #     for state in child:
      #       # print(f'u value before and type- {U[state], type(U[state])}')
      #       U_new[state] = round(reward[state] + gamma * max_prob(state, U, col_size, row_size, obstacle),2)
      #       # print(f'u value and type- {U[state], type(U[state])}')
      #       # print(f'u_new value and type- {U_new[state], type(U_new[state])}')
      #       # print(U_new[state] - U[state])
      #       not_threshold_condition = (not_threshold_condition and (U_new[state] - U[state] < threshold_value))
      #       # print(U_new[state] - U[state], not_threshold_condition)
      # else:
        for state in all_states_order:
          U_new[state] = round(reward[state] + gamma * max_prob(state, U, col_size, row_size, obstacle),2)
          not_threshold_condition = (not_threshold_condition and (U_new[state] - U[state] < threshold_value))

        not_threshold_condition = not (not_threshold_condition)
        U = copy.deepcopy(U_new)
      # not_threshold_condition = not (not_threshold_condition)
      # U = copy.deepcopy(U_new)
    print(f"At what iteration it's stopping {k}")
    return U

In [ ]:
# u1 = valueIteration(0.01, 1, 0.9,reward, U, obstacle, U_new, 4, 3,(4,3),(4,2))
# u1

{(1, 1): 0,
 (1, 2): 0,
 (1, 3): 0,
 (2, 1): 0,
 (2, 2): None,
 (2, 3): 0,
 (3, 1): 0,
 (3, 2): 0,
 (3, 3): 0,
 (4, 1): 0,
 (4, 2): -1,
 (4, 3): 1}

In [ ]:
# u2 = valueIteration(0.01, 2, 0.9,reward, U, obstacle, U_new, 4, 3,(4,3),(4,2))
# u2

{(1, 1): 0,
 (1, 2): 0,
 (1, 3): 0,
 (2, 1): 0,
 (2, 2): None,
 (2, 3): 0.0,
 (3, 1): 0,
 (3, 2): 0.0,
 (3, 3): 0.72,
 (4, 1): 0,
 (4, 2): -1,
 (4, 3): 1}

In [ ]:
# u3 = valueIteration(0.01, 3, 0.9,reward, U, obstacle, U_new, 4, 3,(4,3),(4,2))
# u3

{(1, 1): 0,
 (1, 2): 0,
 (1, 3): 0.0,
 (2, 1): 0,
 (2, 2): None,
 (2, 3): 0.52,
 (3, 1): 0.0,
 (3, 2): 0.43,
 (3, 3): 0.78,
 (4, 1): 0,
 (4, 2): -1,
 (4, 3): 1}

In [ ]:
# u6 = valueIteration(0.01, 6, 0.9,reward, U, obstacle, U_new, 4, 3,(4,3),(4,2))
# u6

At what iteration it's stopping 6


{(1, 1): 0.21,
 (1, 2): 0.42,
 (1, 3): 0.59,
 (2, 1): 0.31,
 (2, 2): None,
 (2, 3): 0.73,
 (3, 1): 0.43,
 (3, 2): 0.56,
 (3, 3): 0.85,
 (4, 1): 0.19,
 (4, 2): -1,
 (4, 3): 1}

In [ ]:
# u7 = valueIteration(0.01, 7, 0.9,reward, U, obstacle, U_new, 4, 3,(4,3),(4,2))
# u7

{(1, 1): 0.35,
 (1, 2): 0.5,
 (1, 3): 0.62,
 (2, 1): 0.37,
 (2, 2): None,
 (2, 3): 0.74,
 (3, 1): 0.45,
 (3, 2): 0.57,
 (3, 3): 0.85,
 (4, 1): 0.24,
 (4, 2): -1,
 (4, 3): 1}

In [ ]:
# u100 = valueIteration(0.01, 100, 0.9,reward, U, obstacle, U_new, 4, 3,(4,3),(4,2))
# u100

At what iteration it's stopping 13


{(1, 1): 0.49,
 (1, 2): 0.57,
 (1, 3): 0.65,
 (2, 1): 0.43,
 (2, 2): None,
 (2, 3): 0.75,
 (3, 1): 0.47,
 (3, 2): 0.57,
 (3, 3): 0.85,
 (4, 1): 0.27,
 (4, 2): -1,
 (4, 3): 1}

In [ ]:
u500 = valueIteration(0.01, 500, 0.9,reward, U, obstacle, U_new, 4, 3,(4,3),(4,2))
u500

At what iteration it's stopping 13


{(1, 1): 0.49,
 (1, 2): 0.57,
 (1, 3): 0.65,
 (2, 1): 0.43,
 (2, 2): None,
 (2, 3): 0.75,
 (3, 1): 0.47,
 (3, 2): 0.57,
 (3, 3): 0.85,
 (4, 1): 0.27,
 (4, 2): -1,
 (4, 3): 1}

1. Here, we can see that for 100th iteration and 500th iteration we got the sme value meaning it didn't run for 500 iterations instead stopped when the threshold value is < 0.01
2. Here the ui represents the ith iteration u100 is 100th iteration when I tested with the iterations from 1,2,3,6,7 and 100 which were commented in the above cells, got the same value as shown in the slides.
3. Hence, we can prove that my algorithm works accurate in accordance with the logic provided by the professor.

#visualisation check

In [ ]:
!pip install tabulate

In [ ]:
from tabulate import tabulate
def visualise_table(row_size=3,col_size=4,input_val=u500):
  table = []
  for row in range(row_size,0,-1):
    new_table_value = []
    for col in range(1,col_size+1):
      if not input_val[(col,row)]:
        input_val[(col,row)] = 'X'
      new_table_value.append(input_val[(col,row)])
    table.append(new_table_value)
  return table, tabulate(table, tablefmt="fancy_grid")


In [ ]:
u_tba, v = visualise_table(3,4,u500)
print(u_tba)
print(v)

[[0.65, 0.75, 0.85, 1], [0.57, 'X', 0.57, -1], [0.49, 0.43, 0.47, 0.27]]
╒══════╤══════╤══════╤═══════╕
│ 0.65 │ 0.75 │ 0.85 │  1    │
├──────┼──────┼──────┼───────┤
│ 0.57 │ X    │ 0.57 │ -1    │
├──────┼──────┼──────┼───────┤
│ 0.49 │ 0.43 │ 0.47 │  0.27 │
╘══════╧══════╧══════╧═══════╛


In [ ]:
def get_arrow(row, col, table, obstacle, destination, hazard):
  directions = [(row,col+1),(row+1,col),(row,col-1),(row-1,col)] #right, down, left, up
  row_size,col_size = len(table), len(table[0])
  # print(f'directions prior{directions}')
  for i in range(len(directions)):
    row_val, col_val = directions[i][0], directions[i][1]
    # table_val = table[row_val][col_val]
    if (row_val >= row_size or col_val >= col_size) or (row_val < 0 or col_val < 0):
      directions[i] = None
    elif (row_val < row_size and col_val < col_size) and (row_val >= 0 and col_val >= 0):
      if (table[row_val][col_val] == 'X'): #or ((col_val+1,len(table)-row_val) == destination) or ((col_val+1,len(table)-row_val) == hazard):
        directions[i] = None
  # print(directions)
  values = []
  for direction in directions:
    if direction:
      row_val, col_val = direction
      # print(f'direction - {row_val,col_val}')
      values.append(table[row_val][col_val])
    else:
      values.append(float('-inf'))
  # print(values)
  max_value = max(values)
  # index_max_val = values.index(max_value)
  index_max_val = [i for i, value in enumerate(values) if value == max_value]
  ans = ''
  for index_val in index_max_val:
    if index_val == 0: #right
      ans += '→'
    elif index_val == 1: #down
      ans += '↓'
    elif index_val == 2: #left
      ans += '←'
    elif index_val == 3: #up
      ans += '↑'
  return ans

def visualise_policy(table=u_tba,destination=D,hazard=H,obstacle1=obstacle):
  arrow = []
  for row in range(len(table)):
    new_arrow = []
    for col in range(len(table[row])):
      if (col+1,len(table)-row) != destination and (col+1,len(table)-row) != hazard and (col+1,len(table)-row) != obstacle1:
        arrow_direction = get_arrow(row,col,table,obstacle1,destination,hazard)
      else:
        arrow_direction = table[row][col]
      new_arrow.append(arrow_direction)
    arrow.append(new_arrow)
  return tabulate(arrow, tablefmt="fancy_grid")


In [ ]:
values = [1,2,2,0]
max_value = max(values)
index_max_val = [i for i, value in enumerate(values) if value == max_value]
index_max_val

[1, 2]

In [ ]:
policy = visualise_policy(u_tba, D, H, obstacle)
print(policy)

╒═══╤═══╤═══╤════╕
│ → │ → │ → │ 1  │
├───┼───┼───┼────┤
│ ↑ │ X │ ↑ │ -1 │
├───┼───┼───┼────┤
│ ↑ │ ← │ ↑ │ ←  │
╘═══╧═══╧═══╧════╛


Here, we can see that our table does interpret same as shown in the slides both for values placement and arrows placement hence we can say that my algorithm works accurately for both values and visualisation.